In [7]:
import torch
from Model.complete_model import CVSeaNet
import matplotlib.pyplot as plt
import rasterio
from IPython.display import clear_output

In [ ]:
path_state = r"Weights\epoch=64-step=134160.ckpt"
state = torch.load(path_state)
in_channels = in_channels = state['hyper_parameters']['in_channels']
in_res = state['hyper_parameters']['in_res']
backbone_type_id = state['hyper_parameters']['backbone_type_id']
backbone_architecture = state['hyper_parameters']['backbone_architecture']
score_threshold = state['hyper_parameters']['score_threshold']
real_conv_block = state['hyper_parameters']['real_conv_block']
model = CVSeaNet(in_channels=in_channels, in_resolution=in_res, backbone_params=[backbone_type_id, backbone_architecture],
                 score_threshold=score_threshold, real_conv_block=real_conv_block).to(device='cuda', dtype=torch.float16)
state_dict_without_prefix = {key.replace('model.', ''): value for key, value in state['state_dict'].items()}     
model.load_state_dict(state_dict_without_prefix)
clear_output(True)

file_path = input("Insert image path (.tif)")
with rasterio.open(file_path) as f:
    x_real_vh = f.read(1)
    x_imag_vh = f.read(2)
x_real_vh = torch.from_numpy(x_real_vh)
x_imag_vh = torch.from_numpy(x_imag_vh)
img = torch.stack((x_real_vh, x_imag_vh))
img = img.unsqueeze(0).to(device='cuda', dtype=torch.float16)


model.eval()
out = model(img)
idx_kp = torch.nonzero(out['Keypoints heatmap'] > score_threshold)
idx_kp = idx_kp[:, 2:]
off = out['Offset heatmap']
x_off = off[0,0,idx_kp[:,0], idx_kp[:,1]].unsqueeze(-1)
y_off = off[0,1,idx_kp[:,0], idx_kp[:,1]].unsqueeze(-1)
off = torch.cat((x_off, y_off),dim=-1)
scaled_out = idx_kp + off
scaled_out = scaled_out * out['Scale Parameter']
plt.figure(figsize=(20,20))
plt.imshow((torch.abs(img[0,0].to(torch.float) + 1j * img[0,1].to(torch.float)).cpu().numpy()), cmap='gray')
plt.scatter(scaled_out[:,0].to(torch.float).detach().cpu().numpy(), scaled_out[:,1].to(torch.float).detach().cpu().numpy(), marker='x', c='blue')